# 自定义层

构造一个没有任何参数的自定义层

In [1]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

将层作为组件合并到构建更复杂的模型中

In [2]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(1.8626e-09, grad_fn=<MeanBackward0>)

带参数的层

In [9]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        
        self.weight = nn.Parameter(torch.randn(in_units, units)) # 初始化并添加的nn.Parameter中，也就是让计算梯度等
        self.bias = nn.Parameter(torch.randn(units))
        
    def forward(self, X):
        linear = torch.mm(X, self.weight.data) + self.bias.data # 是计算梯度的张量，计算时需要取出数据
        return F.relu(linear)

linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 1.1803,  0.2318,  0.2378],
        [ 0.0197, -1.7643,  0.2707],
        [-1.1243, -0.9125, -0.8266],
        [-0.8486,  0.3383,  0.9031],
        [-1.6054, -0.3678,  0.7515]], requires_grad=True)

使用自定义层直接执行正向传播计算

In [10]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 0.4163],
        [0.0000, 0.0000, 0.2836]])

使用自定义层构建模型

In [11]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])